## Analyze Petrel reference data using [Graphs](https://en.wikipedia.org/wiki/Graph_(discrete_mathematics%29) in Python [Networkx](https://networkx.github.io/documentation/stable/index.html)


This notebook gives an example of how Python and the Networkx package can be used to apply Graph and network analysis to explore Petrel project data. The goal is to analyze to what degree the projects are consistent for use with Petrel reference projects and Studio repositories.

The analysis is based on data collected and structured by [Blueback Project Tracker](https://cegal.com/products/oil-gas-workspace/data-management-products/blueback-project-tracker/) from Cegal.

For an introduction to reference projects, search for "reference project" and "guid" in the Petrel user manual.

For an introduction to network analysis in Python see this book: [Complex Network Analysis in Python](http://a.co/2SXQAl6) by Dmitry Zinoviev, and the [Networkx documentation](https://networkx.github.io/documentation/stable/index.html)


### Read data fromt the Blueback Project Tracker MS SQL database

Change the server and database

In [149]:
import pandas as pd
import pymssql

sql_server = 'YourServer'      # <<<<<<<<<<< Change this to your SQL Server instance
sql_database = 'Tracker'       # <<<<<<<<<<< Change this to your Tracker database name

# Using Windows authentication
con = pymssql.connect(server=sql_server, database=sql_database)
sql = 'SELECT ProjectId, Path, GuidString, DataName FROM DataDetailed WHERE DataTypeId=9'
df = pd.read_sql(sql, con)
df.columns = ['ProjectId', 'Path', 'Guid', 'WellName']

df.head()

,ProjectId,Path,Guid,WellName
0,3,D:\NotBackedup\2012.pet,02C62C82-552D-444D-BF6B-69CD07376368,A10
1,3,D:\NotBackedup\2012.pet,6B5A07A9-695C-4340-B3B3-E5F24C61DA85,A15
2,3,D:\NotBackedup\2012.pet,EA7DE57F-F769-4952-810F-A125DC93E9FE,A16
3,4,D:\NotBackedup\2012_2014.petR,02C62C82-552D-444D-BF6B-69CD07376368,A10
4,4,D:\NotBackedup\2012_2014.petR,6B5A07A9-695C-4340-B3B3-E5F24C61DA85,A15


### Create the graph by creating edges between projects and data (wells). 

The nodes in the network will represent the Petrel projects and the data items. There will be an edge between each project and all the data items in that projects. If one data item exists in more than one project (the data item has the same GUID in all projects) there will be multiple edges in to that data item node.

The degree of a project node will tell how many data items there are in that project.
The degree of a data node will tell how many projects contains this data item.

Note that duplicate nodes will not be created.

In [119]:
import networkx as nx

G = nx.from_pandas_edgelist(df, source='ProjectId', target='Guid')
print(nx.info(G))
print('Number of wells in Project with id 3: {}'.format(nx.degree(G, 3)))
print('Number of projects where well A10 with GUID 02C62C82-552D-444D-BF6B-69CD07376368 exists: {}'.format(nx.degree(G, '02C62C82-552D-444D-BF6B-69CD07376368')))

Name: 
Type: Graph
Number of nodes: 20329
Number of edges: 62616
Average degree:   6.1603
Number of wells in Project with id 3: 3
Number of projects where well A10 with GUID 02C62C82-552D-444D-BF6B-69CD07376368 exists: 47


Extract the project nodes and data nodes. Set the bipartite attribute. This can be used later for some of the bipartite algorithms.


In [151]:
from networkx.algorithms import bipartite

projectIds = df['ProjectId'].values
guids = df['Guid'].values
p = dict.fromkeys(projectIds, 0)
g = dict.fromkeys(guids, 1)
nx.set_node_attributes(G, p, 'bipartite')
nx.set_node_attributes(G, g, 'bipartite')

project_nodes = {n for n, d in G.nodes(data=True) if d['bipartite']==0}
data_nodes = set(G) - project_nodes

print('Number of project nodes: {}'.format(len(project_nodes)))
print('Number of date nodes: {}'.format(len(data_nodes)))

Number of project nodes: 139
Number of date nodes: 20190


The degree of the project nodes is the number of wells in the projects

In [122]:
project_degrees = nx.degree(G, nbunch=project_nodes)
# Set the number of wells attribute on the project nodes
nx.set_node_attributes(G, dict(project_degrees), 'NumberOfWells')


### Extract project network

The graph of projects and data is a [bipartite graph](https://en.wikipedia.org/wiki/Bipartite_graph). With a bipartite graph we can project the project nodes and extract a new graph with only projects. The projects will be connected by edges based on the wells they share. A weight on the edge will be equal to the number of wells thee projects have in common.

If there are more than one connected component in the graph, this indicates that there are subsets of projects that do not share data items based on GUID. This can happen if data items are imported into different projects without using reference projects or Studio repositories.

In [127]:
PG = bipartite.weighted_projected_graph(G, project_nodes)

print('Number of connected componenets: {}'.format(nx.number_connected_components(PG)))
print('Number of projects: {}'.format(df['ProjectId'].nunique()))
print('Number of project nodes: {}'.format(len(PG.nodes())))


Number of connected componenets: 44
Number of projects: 139
Number of project nodes: 139


### Rank the projects to find potential reference projects

Node [centrality](https://en.wikipedia.org/wiki/Centrality) is a way to identify important or central nodes.

In the Project graph the degree of projects nodes gives us the number of projects that share data items with that project.
The weight of the edges between projects gives us the number of data items shared between the projects.
(See [this](https://toreopsahl.com/tnet/weighted-networks/node-centrality/) article for a discussion)

We want to find the projects that contains the most wells that are shared with other projects. One way to do this is to calculate a rank as a product of the degree and weighted degree and number of data items in the project.




In [142]:
ref_project_rank = ([(n, nx.degree(PG, nbunch=n)*nx.degree(PG, nbunch=n, weight='weight')*((PG.nodes[n]['NumberOfWells']))) for n in PG.nodes() ])
# Set the rank as an attribute on the nodes
nx.set_node_attributes(PG, dict(ref_project_rank), 'Rank')

For each project we now have the NumberOfWells and Rank as node attributes:

In [143]:
list(PG.nodes.data())[:5]

[(10496, {'NumberOfWells': 163, 'Rank': 0, 'bipartite': 0}),
 (257, {'NumberOfWells': 155, 'Rank': 10643385, 'bipartite': 0}),
 (10498, {'NumberOfWells': 245, 'Rank': 0, 'bipartite': 0}),
 (3, {'NumberOfWells': 3, 'Rank': 17907, 'bipartite': 0}),
 (4, {'NumberOfWells': 3, 'Rank': 17907, 'bipartite': 0})]

If all well data exists in a common reference project the whole graph will be connected. If no common Guids exists between projects the number of disconnected sub-graphs will be equal to the number of projects.


In [144]:
print('Are all projects connected?: {}'.format(nx.is_connected(G)))
sub_graphs = list(nx.connected_component_subgraphs(PG))
print('Number of disconnected sets of Petrel projects: {}'.format(len(sub_graphs)))

Are all projects connected?: False
Number of disconnected sets of Petrel projects: 44


Sort the sub graphs from largest to smallest. Print size of top 10. Small sets are usually because of project duplications after Save As new project in Petrel.

In [145]:
sorted_sub_graphs = sorted(sub_graphs, key=len, reverse=True)
l = [len(sg) for sg in sorted_sub_graphs]
l[:10]

[63, 11, 9, 6, 3, 3, 2, 2, 2, 2]

Plot the largest graph. ProjectId as label.

In [161]:
#%matplotlib inline
#import matplotlib.pyplot as plt

#f = plt.figure(figsize=(10,10))
#ax = f.add_subplot(121)
#nx.draw(sorted_sub_graphs[0], with_labels=True)
#plt.show()

### List projects that can be used as reference projects or to populate Studio repositories

The reference project is most likely the project that has the highest rank.
For each set of projects, sort by rank and list the projects from highest to lower rank. The top most projects are candidates for reference projects or Studio repositories.

The [density](https://en.wikipedia.org/wiki/Dense_graph) of the group is a measure for how well connected the projects are. If 1.0 all projects share a common set of data.


In [162]:

for g in sorted_sub_graphs:
    if len(g) < 4: # Skip small groups
        continue
    print('Size of group: {}'.format(len(g)))
    print('Density of group: ', nx.density(g))
    
    # sort based on rank from highest to lowest rank
    g_rank = list(g.nodes.data())
    g_sorted = sorted(g_rank, key=lambda tup : tup[1]['Rank'], reverse=True)
    
    max_rank = g_sorted[0][1]['Rank']
    if (max_rank == 0):
        max_rank = 1
    for sbtw in g_sorted[:5]:
        pid = sbtw[0]
        r = sbtw[1]['Rank']/max_rank # Normalize the rank to be between 0.0 and 1.0
        nw = sbtw[1]['NumberOfWells']
        project_path = df.loc[df['ProjectId'] == pid]['Path'].head(1).values[0]

        print('\t {0}\t{1:.4f}\t{2}\t{3}'.format(pid, r, nw, project_path))


Size of group: 63
Density of group:  0.5934459805427548
	 10507	1.0000	3504	\\it19-ta-013\G16
	 50588	0.0210	3501	\\it19-ta-016\Studio2018\LoadTest
	 257	0.0182	155	D:\NotBackedup\GullfaksDemoDataset_July24_2014\Petrel2014_1 demo project.pet
	 10443	0.0182	155	D:\NotBackedup\TestTracker601.pet
	 337	0.0182	155	D:\NotBackedup\Petrel2014Gullfaks23Dec2014\Petrel 2014 Gullfaks 23Dec2014\Petrel2015 demo project ref.pet
Size of group: 11
Density of group:  1.0
	 63	1.0000	1496	D:\NotBackedup\npdStudio2.pet
	 220	1.0000	1496	D:\NotBackedup\DemoMove\npdexpl2015.pet
	 222	1.0000	1496	D:\NotBackedup\DemoMove\npdStudio2.pet
	 265	1.0000	1496	D:\NotBackedup\IntericaProjects\npdexpl.pet
	 412	1.0000	1496	D:\NotBackedup\Studio Test\Project2-2014.pet
Size of group: 9
Density of group:  1.0
	 401	1.0000	3	D:\NotBackedup\SpatialTest\WellReference.pet
	 370	1.0000	3	D:\NotBackedup\RefDataTest\WellDataTest\WellReferenceProject.petR
	 371	1.0000	3	D:\NotBackedup\RefDataTest\WellDataTestOld\UserProject1.pe